In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset = pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [4]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [5]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [6]:
dataset.drop('User ID',axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
indep = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [8]:
dep = dataset['Purchased']
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep,test_size=1/3,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy', 'log_loss'],'max_features':['sqrt', 'log2', None],
              'class_weight':['balanced', 'balanced_subsample'], 'min_samples_split':[1,2]}

grid = GridSearchCV (RandomForestClassifier(), param_grid, refit = True, verbose = 3, n_jobs = -1, scoring = 'f1_weighted')
grid.fit(X_train,Y_train)


Fitting 5 folds for each of 36 candidates, totalling 180 fits


C:\Users\Hxtreme\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Hxtreme\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Hxtreme\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\Hxtreme\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Hxtreme\anaconda3\Lib\sit

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced', 'balanced_subsample'],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_split': [1, 2]},
             scoring='f1_weighted', verbose=3)

In [33]:
#print best parameter after tuning
#print(grid.best_params_)
re = grid.cv_results_
grid_predictions = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,grid_predictions)

In [38]:
print(cm)

[[79  6]
 [ 4 45]]


In [34]:
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test,grid_predictions)

In [39]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        85
           1       0.88      0.92      0.90        49

    accuracy                           0.93       134
   macro avg       0.92      0.92      0.92       134
weighted avg       0.93      0.93      0.93       134



In [ ]:
from sklearn.metrics import f1_score
f1_macro = f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)

In [41]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9631452581032414

In [42]:
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_features,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003200,7.481642e-04,0.000000,0.000000,balanced,gini,sqrt,1,"{'class_weight': 'balanced', 'criterion': 'gin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
1,0.522166,1.910189e-02,0.027998,0.001095,balanced,gini,sqrt,2,"{'class_weight': 'balanced', 'criterion': 'gin...",0.867478,0.888107,0.851527,0.925272,0.962264,0.898930,0.040146,4
2,0.003000,2.000070e-03,0.000000,0.000000,balanced,gini,log2,1,"{'class_weight': 'balanced', 'criterion': 'gin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
3,0.486371,1.121734e-02,0.029598,0.005199,balanced,gini,log2,2,"{'class_weight': 'balanced', 'criterion': 'gin...",0.867478,0.888107,0.851527,0.925272,0.962264,0.898930,0.040146,4
4,0.001999,1.095932e-03,0.000000,0.000000,balanced,gini,None,1,"{'class_weight': 'balanced', 'criterion': 'gin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
5,0.496769,7.572823e-03,0.026598,0.000800,balanced,gini,None,2,"{'class_weight': 'balanced', 'criterion': 'gin...",0.867478,0.868752,0.851527,0.887907,0.943041,0.883741,0.031814,13
6,0.002000,4.909339e-07,0.000000,0.000000,balanced,entropy,sqrt,1,"{'class_weight': 'balanced', 'criterion': 'ent...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
7,0.505969,1.744538e-02,0.029198,0.002638,balanced,entropy,sqrt,2,"{'class_weight': 'balanced', 'criterion': 'ent...",0.867478,0.907177,0.851527,0.925272,0.943041,0.898899,0.034479,8
8,0.001601,4.898047e-04,0.000000,0.000000,balanced,entropy,log2,1,"{'class_weight': 'balanced', 'criterion': 'ent...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
9,0.593008,3.302777e-02,0.035754,0.005507,balanced,entropy,log2,2,"{'class_weight': 'balanced', 'criterion': 'ent...",0.847141,0.907177,0.851527,0.925272,0.943699,0.894963,0.039030,9


In [43]:
age_input = float(input("Age:"))
salary_input = float(input("Salary:"))
sex_male_input = int(input("sex_male 0 or 1:"))
Future_Prediction = grid.predict([[age_input, salary_input, sex_male_input]])
Future_Prediction

Age: 23
Salary: 43545
sex_male 0 or 1: 1


array([1], dtype=int64)